In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16 ,VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir="/content/drive/MyDrive/spam image datset/SPAM IMAGE dataset2/train"
validation_dir="/content/drive/MyDrive/spam image datset/SPAM IMAGE dataset2/val"

In [ ]:
img_size = (224, 224)
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 1235 images belonging to 2 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 359 images belonging to 2 classes.


In [ ]:
# Load pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Combine base model with new classification layers
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
38/38 [==============================] - 1098s 29s/step - loss: 0.1902 - accuracy: 0.9518 - val_loss: 0.0690 - val_accuracy: 0.9744
Epoch 2/10
38/38 [==============================] - 1081s 29s/step - loss: 0.0750 - accuracy: 0.9800 - val_loss: 0.0597 - val_accuracy: 0.9858
Epoch 3/10
38/38 [==============================] - 1073s 28s/step - loss: 0.0703 - accuracy: 0.9834 - val_loss: 0.0464 - val_accuracy: 0.9886
Epoch 4/10
38/38 [==============================] - 1069s 28s/step - loss: 0.0625 - accuracy: 0.9842 - val_loss: 0.0510 - val_accuracy: 0.9858
Epoch 5/10
38/38 [==============================] - 1070s 28s/step - loss: 0.0587 - accuracy: 0.9817 - val_loss: 0.0479 - val_accuracy: 0.9858
Epoch 6/10
38/38 [==============================] - 1047s 28s/step - loss: 0.0527 - accuracy: 0.9850 - val_loss: 0.0308 - val_accuracy: 0.9943
Epoch 7/10
38/38 [==============================] - 1047s 28s/step - loss: 0.0483 - accuracy: 0.9875 - val_loss: 0.0429 - val_accuracy: 0.9915

In [ ]:
from tensorflow.keras.preprocessing import image
# Function to preprocess uploaded image,
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
import numpy as np
# Function to predict whether the image contains spam or not
def predict_spam(image_path):
    processed_img = preprocess_image(image_path)
    prediction = model.predict(processed_img)
    return prediction[0][0]  # Return the prediction value (probability)

# Provide the path to the uploaded image for prediction
uploaded_image_path = '/content/drive/MyDrive/New folder (1)/dredze/personal_image_spam/000001c723a9$15785000$0100007f@localhost_image90.jpg'

# Predict whether the uploaded image contains spam
prediction_result = predict_spam(uploaded_image_path)

# Threshold for considering if it's spam or not (adjust as needed)
spam_threshold = 0.5

if prediction_result >= spam_threshold:
    print("The uploaded image is predicted as SPAM.")
else:
    print("The uploaded image is predicted as NOT SPAM.")


1/1 [==============================] - 1s 805ms/step
The uploaded image is predicted as NOT SPAM.


# **pretrained models**

In [18]:
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3, ResNet50, MobileNetV2, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
import tensorflow as tf

In [20]:
def create_model(base_model, img_size=(224, 224), num_classes=1):
    # Add new classification layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='sigmoid')(x)

    # Combine base model with new classification layers
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze layers of the pre-trained model
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Define a function to train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size
    )
    return history

# Function to load a saved model
def load_saved_model(model_path):
    model = tf.keras.models.load_model(model_path)
    return model

# Example usage:

# Define your dataset directory paths
train_dir = '/content/drive/MyDrive/spam image datset/SPAM IMAGE dataset2/train'
validation_dir = '/content/drive/MyDrive/spam image datset/SPAM IMAGE dataset2/val'

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 1235 images belonging to 2 classes.
Found 359 images belonging to 2 classes.


In [4]:
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg16 = create_model(base_model_vgg16)

58889256/58889256 [==============================] - 0s 0us/step


In [5]:
history_vgg16 = train_model(model_vgg16, train_generator, validation_generator)


Epoch 1/10
38/38 [==============================] - 307s 8s/step - loss: 0.1909 - accuracy: 0.9601 - val_loss: 0.0647 - val_accuracy: 0.9886
Epoch 2/10
38/38 [==============================] - 20s 534ms/step - loss: 0.0733 - accuracy: 0.9809 - val_loss: 0.0666 - val_accuracy: 0.9801
Epoch 3/10
38/38 [==============================] - 21s 554ms/step - loss: 0.0721 - accuracy: 0.9834 - val_loss: 0.0491 - val_accuracy: 0.9886
Epoch 4/10
38/38 [==============================] - 19s 507ms/step - loss: 0.0696 - accuracy: 0.9834 - val_loss: 0.0489 - val_accuracy: 0.9915
Epoch 5/10
38/38 [==============================] - 22s 588ms/step - loss: 0.0572 - accuracy: 0.9842 - val_loss: 0.0471 - val_accuracy: 0.9915
Epoch 6/10
38/38 [==============================] - 20s 517ms/step - loss: 0.0557 - accuracy: 0.9834 - val_loss: 0.0447 - val_accuracy: 0.9915
Epoch 7/10
38/38 [==============================] - 22s 565ms/step - loss: 0.0465 - accuracy: 0.9875 - val_loss: 0.0435 - val_accuracy: 0.9915
E

In [7]:
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg19 = create_model(base_model_vgg19)

80134624/80134624 [==============================] - 0s 0us/step


In [8]:
history_vgg19 = train_model(model_vgg19, train_generator, validation_generator)

Epoch 1/10
38/38 [==============================] - 23s 539ms/step - loss: 0.2463 - accuracy: 0.9460 - val_loss: 0.0870 - val_accuracy: 0.9830
Epoch 2/10
38/38 [==============================] - 20s 532ms/step - loss: 0.0845 - accuracy: 0.9767 - val_loss: 0.0635 - val_accuracy: 0.9858
Epoch 3/10
38/38 [==============================] - 22s 568ms/step - loss: 0.0728 - accuracy: 0.9776 - val_loss: 0.0886 - val_accuracy: 0.9688
Epoch 4/10
38/38 [==============================] - 21s 546ms/step - loss: 0.0739 - accuracy: 0.9809 - val_loss: 0.0605 - val_accuracy: 0.9830
Epoch 5/10
38/38 [==============================] - 21s 562ms/step - loss: 0.0609 - accuracy: 0.9825 - val_loss: 0.0567 - val_accuracy: 0.9830
Epoch 6/10
38/38 [==============================] - 20s 536ms/step - loss: 0.0615 - accuracy: 0.9784 - val_loss: 0.0561 - val_accuracy: 0.9801
Epoch 7/10
38/38 [==============================] - 21s 553ms/step - loss: 0.0535 - accuracy: 0.9842 - val_loss: 0.0486 - val_accuracy: 0.9886

In [10]:
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_resnet50 = create_model(base_model_resnet50)

94765736/94765736 [==============================] - 0s 0us/step


In [11]:
history_resnet50 = train_model(model_resnet50, train_generator, validation_generator)

Epoch 1/10
38/38 [==============================] - 28s 544ms/step - loss: 0.5522 - accuracy: 0.7648 - val_loss: 0.4217 - val_accuracy: 0.8352
Epoch 2/10
38/38 [==============================] - 20s 512ms/step - loss: 0.3824 - accuracy: 0.8603 - val_loss: 0.3206 - val_accuracy: 0.8949
Epoch 3/10
38/38 [==============================] - 19s 501ms/step - loss: 0.2937 - accuracy: 0.9185 - val_loss: 0.2699 - val_accuracy: 0.9318
Epoch 4/10
38/38 [==============================] - 21s 553ms/step - loss: 0.2525 - accuracy: 0.9235 - val_loss: 0.2642 - val_accuracy: 0.9062
Epoch 5/10
38/38 [==============================] - 20s 513ms/step - loss: 0.2312 - accuracy: 0.9393 - val_loss: 0.2719 - val_accuracy: 0.8949
Epoch 6/10
38/38 [==============================] - 20s 511ms/step - loss: 0.2206 - accuracy: 0.9360 - val_loss: 0.2740 - val_accuracy: 0.8835
Epoch 7/10
38/38 [==============================] - 19s 505ms/step - loss: 0.1898 - accuracy: 0.9485 - val_loss: 0.2068 - val_accuracy: 0.9460

In [13]:
base_model_inceptionv3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_inceptionv3 = create_model(base_model_inceptionv3)

87910968/87910968 [==============================] - 0s 0us/step


In [14]:
history_inceptionv3 = train_model(model_inceptionv3, train_generator, validation_generator)

Epoch 1/10
38/38 [==============================] - 30s 611ms/step - loss: 0.1341 - accuracy: 0.9568 - val_loss: 0.0936 - val_accuracy: 0.9716
Epoch 2/10
38/38 [==============================] - 19s 494ms/step - loss: 0.0300 - accuracy: 0.9917 - val_loss: 0.0746 - val_accuracy: 0.9744
Epoch 3/10
38/38 [==============================] - 19s 488ms/step - loss: 0.0246 - accuracy: 0.9917 - val_loss: 0.0834 - val_accuracy: 0.9773
Epoch 4/10
38/38 [==============================] - 20s 538ms/step - loss: 0.0174 - accuracy: 0.9942 - val_loss: 0.0771 - val_accuracy: 0.9801
Epoch 5/10
38/38 [==============================] - 18s 477ms/step - loss: 0.0116 - accuracy: 0.9967 - val_loss: 0.0843 - val_accuracy: 0.9773
Epoch 6/10
38/38 [==============================] - 20s 523ms/step - loss: 0.0107 - accuracy: 0.9958 - val_loss: 0.1029 - val_accuracy: 0.9801
Epoch 7/10
38/38 [==============================] - 18s 485ms/step - loss: 0.0091 - accuracy: 0.9975 - val_loss: 0.0853 - val_accuracy: 0.9744

In [16]:
base_model_mobilenetv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_mobilenetv2 = create_model(base_model_mobilenetv2)

9406464/9406464 [==============================] - 0s 0us/step


In [17]:
history_mobilenetv2 = train_model(model_mobilenetv2, train_generator, validation_generator)

Epoch 1/10
38/38 [==============================] - 25s 542ms/step - loss: 0.1282 - accuracy: 0.9634 - val_loss: 0.0654 - val_accuracy: 0.9801
Epoch 2/10
38/38 [==============================] - 18s 482ms/step - loss: 0.0396 - accuracy: 0.9892 - val_loss: 0.0470 - val_accuracy: 0.9886
Epoch 3/10
38/38 [==============================] - 18s 482ms/step - loss: 0.0234 - accuracy: 0.9910 - val_loss: 0.0396 - val_accuracy: 0.9886
Epoch 4/10
38/38 [==============================] - 20s 519ms/step - loss: 0.0182 - accuracy: 0.9950 - val_loss: 0.0376 - val_accuracy: 0.9886
Epoch 5/10
38/38 [==============================] - 18s 477ms/step - loss: 0.0101 - accuracy: 0.9975 - val_loss: 0.0496 - val_accuracy: 0.9858
Epoch 6/10
38/38 [==============================] - 18s 479ms/step - loss: 0.0084 - accuracy: 0.9992 - val_loss: 0.0369 - val_accuracy: 0.9915
Epoch 7/10
38/38 [==============================] - 18s 479ms/step - loss: 0.0071 - accuracy: 0.9983 - val_loss: 0.0384 - val_accuracy: 0.9886

In [21]:
base_model_densenet50 = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_densenet50 = create_model(base_model_densenet50)

29084464/29084464 [==============================] - 0s 0us/step


In [22]:
history_densenet50 = train_model(model_densenet50, train_generator, validation_generator)

Epoch 1/10
38/38 [==============================] - 33s 598ms/step - loss: 0.1213 - accuracy: 0.9518 - val_loss: 0.0621 - val_accuracy: 0.9830
Epoch 2/10
38/38 [==============================] - 20s 532ms/step - loss: 0.0303 - accuracy: 0.9909 - val_loss: 0.0526 - val_accuracy: 0.9830
Epoch 3/10
38/38 [==============================] - 20s 530ms/step - loss: 0.0178 - accuracy: 0.9958 - val_loss: 0.0752 - val_accuracy: 0.9801
Epoch 4/10
38/38 [==============================] - 19s 502ms/step - loss: 0.0188 - accuracy: 0.9917 - val_loss: 0.0569 - val_accuracy: 0.9801
Epoch 5/10
38/38 [==============================] - 20s 532ms/step - loss: 0.0135 - accuracy: 0.9967 - val_loss: 0.0550 - val_accuracy: 0.9801
Epoch 6/10
38/38 [==============================] - 19s 496ms/step - loss: 0.0100 - accuracy: 0.9983 - val_loss: 0.0567 - val_accuracy: 0.9801
Epoch 7/10
38/38 [==============================] - 19s 495ms/step - loss: 0.0076 - accuracy: 0.9983 - val_loss: 0.0521 - val_accuracy: 0.9801

In [62]:
# model_vgg16.save('spam_image_filter_model_vgg16.h5')
# model_vgg19.save('spam_image_filter_model_vgg19.h5')
model_resnet50.save('spam_image_filter_model_resnet50.h5')
# model_inceptionv3.save('spam_image_filter_model_inceptionv3.h5')
# model_mobilenetv2.save('spam_image_filter_model_mobilenetv2.h5')
# model_densenet50.save('spam_image_filter_model_densenet50.h5')


In [69]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('spam_image_filter_model_resnet50.h5')



# Function to preprocess uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to predict whether the image contains spam or not

def predict_spam(image_path):
    processed_img = preprocess_image(image_path)
    prediction = model.predict(processed_img)
    return prediction[0][0]  # Return the prediction value (probability)

# Provide the path to the uploaded image for prediction
uploaded_image_path = '/content/drive/MyDrive/spam image datset/SPAM IMAGE dataset2/test/SpamImages/6G6Vx4rm6j.jpg'

# Predict whether the uploaded image contains spam
prediction_result = predict_spam(uploaded_image_path)

# Threshold for considering if it's spam or not (adjust as needed)
spam_threshold = 0.5

if prediction_result >= spam_threshold:
    print("The uploaded image is predicted as SPAM.")
else:
    print("The uploaded image is predicted as NOT SPAM.")

1/1 [==============================] - 1s 877ms/step
The uploaded image is predicted as SPAM.
